---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [3]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [23]:
def answer_one():
    return nx.read_edgelist('Employee_Movie_Choices.txt', delimiter='\t')

G = answer_one()
print("EDGES: {}".format(G.number_of_edges()))
for edge in G.edges():
    print(edge)

print("\nNODES: {}".format(G.number_of_nodes()))
for node in G.nodes():
    print(node)

EDGES: 24
('Andy', 'Anaconda')
('Andy', 'Mean Girls')
('Andy', 'The Matrix')
('Anaconda', 'Claude')
('Anaconda', 'Georgia')
('Mean Girls', 'Joan')
('Mean Girls', 'Lee')
('The Matrix', 'Frida')
('The Matrix', 'Pablo')
('Claude', 'Monty Python and the Holy Grail')
('Claude', 'Snakes on a Plane')
('Monty Python and the Holy Grail', 'Georgia')
('Snakes on a Plane', 'Georgia')
('Frida', 'The Shawshank Redemption')
('Frida', 'The Social Network')
('The Shawshank Redemption', 'Pablo')
('The Shawshank Redemption', 'Vincent')
('The Social Network', 'Vincent')
('Joan', 'Forrest Gump')
('Joan', 'Kung Fu Panda')
('Forrest Gump', 'Lee')
('Kung Fu Panda', 'Lee')
('Pablo', 'The Dark Knight')
('Vincent', 'The Godfather')

NODES: 19
Andy
Anaconda
Mean Girls
The Matrix
Claude
Monty Python and the Holy Grail
Snakes on a Plane
Frida
The Shawshank Redemption
The Social Network
Georgia
Joan
Forrest Gump
Kung Fu Panda
Lee
Pablo
The Dark Knight
Vincent
The Godfather


### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [24]:
def answer_two():
    
    G = answer_one()
    for emp in employees:
        G.node[emp]['type'] = 'employee'
    for movie in movies:
        G.node[movie]['type'] = 'movie'
    
    return G

for node in answer_two().nodes(data=True):
    print(node)

('Andy', {'type': 'employee'})
('Anaconda', {'type': 'movie'})
('Mean Girls', {'type': 'movie'})
('The Matrix', {'type': 'movie'})
('Claude', {'type': 'employee'})
('Monty Python and the Holy Grail', {'type': 'movie'})
('Snakes on a Plane', {'type': 'movie'})
('Frida', {'type': 'employee'})
('The Shawshank Redemption', {'type': 'movie'})
('The Social Network', {'type': 'movie'})
('Georgia', {'type': 'employee'})
('Joan', {'type': 'employee'})
('Forrest Gump', {'type': 'movie'})
('Kung Fu Panda', {'type': 'movie'})
('Lee', {'type': 'employee'})
('Pablo', {'type': 'employee'})
('The Dark Knight', {'type': 'movie'})
('Vincent', {'type': 'employee'})
('The Godfather', {'type': 'movie'})


### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [25]:
def answer_three():
    G = answer_two()
    return bipartite.weighted_projected_graph(G, employees)

for edge in answer_three().edges(data=True):
    print(edge)

('Andy', 'Claude', {'weight': 1})
('Andy', 'Lee', {'weight': 1})
('Andy', 'Joan', {'weight': 1})
('Andy', 'Pablo', {'weight': 1})
('Andy', 'Georgia', {'weight': 1})
('Andy', 'Frida', {'weight': 1})
('Claude', 'Georgia', {'weight': 3})
('Lee', 'Joan', {'weight': 3})
('Pablo', 'Vincent', {'weight': 1})
('Pablo', 'Frida', {'weight': 2})
('Vincent', 'Frida', {'weight': 2})


### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [40]:
def answer_four():
    common_movies = answer_three()
    relationships = nx.read_edgelist("Employee_Relationships.txt", delimiter="\t", data=(('weight', float),))
    
    # I don't know a fancy way to pull out the weights and get them in a form I want.
    # nx.to_pandas_adjacency() got me part of the way there, but it gave me a 
    # num_employees x num_employees dataframe, with zeros on the diagonal. I would have
    # had a tough time figuring out how to mangle that into the form I need.
    # So I took the simple approach here: just iterate through all the pairs
    # of employees, get the weights from the two graphs, and put all of them into lists.
    # Then use those lists to make a dataframe. It's not elegant, but it works.
    relationship_data = []
    movie_data = []
    index = []
    for person1, person2, relationship in relationships.edges(data=True):
        index.append((person1, person2))
        relationship_data.append(relationship['weight'])
        movie_data.append(common_movies[person1][person2]['weight'] if common_movies.has_edge(person1, person2) else 0)
    
    df = pd.DataFrame({'relationship': relationship_data, 'common_movies': movie_data}, index=index)
    
    return df.corr()
answer_four()

,relationship,common_movies
relationship,1.000000,0.788396
common_movies,0.788396,1.000000
